In [1]:
import javalang
import pandas as pd
# from func import *

In [464]:
data=pd.read_csv('complexity_data/finalFeatureData.csv')
file_data=list(data['file_name'])

In [586]:
for f_n in file_data:
    if not f_n in ['1113.java']:
        f = open('complexity_data/origin/' + f_n)
        source = f.read()
        f.close()
        print(f_n)
        tree = javalang.parse.parse(source)
        if not equal_test(tree):
            print(f_n)
            break


1001.java
1100.java
1101.java
1102.java
1103.java
1104.java
1105.java
1106.java
1107.java
1108.java
1109.java
1110.java
1111.java
1112.java
1114.java
1115.java
length miss match!
1115.java


In [36]:
sample=[]

for _,node in tree:
    if type(node)== javalang.tree.IfStatement:
        print(node)
        sample.append(node)
        

In [634]:
f = open('complexity_data/origin/1115.java')
source = f.read()
f.close()
tree=javalang.parse.parse(source)
root=tree.__iter__().__next__()[1]

In [660]:
func_dict={"ClassCreator":ClassCreator2Code,
"Literal":Literal2Code,
    'BinaryOperation':BinaryOperation2Code,
    'MemberReference':MemberReference2Code,
    'BlockStatement':BlockStatement2Code,
    'LocalVariableDeclaration':LocalVariableDeclaration2Code,
    'VariableDeclaration':VariableDeclaration2Code,
    'VariableDeclarator':VariableDeclarator2Code,
    'ArrayCreator': ArrayCreator2Code,
    'StatementExpression':StatementExpression2Code,
    'ArraySelector':ArraySelector2Code,
    'MethodInvocation':MethodInvocation2Code,
    'Assignment':Assignment2Code,
    'IfStatement':IfStatement2Code,
    'ForStatement':ForStatement2Code,
    'MethodDeclaration':MethodDeclaration2Code,
    'Cast':Cast2Code,
    'ReturnStatement':ReturnStatement2Code,
    'ClassDeclaration':ClassDeclaration2Code,
    'FieldDeclaration':FieldDeclaration2Code,
    'NoneType':Nonetype,
    'ConstructorDeclaration':ConstructorDeclaration2Code,
    'WhileStatement':WhileStatement2Code,
    'TryStatement':TryStatement2Code,
    'CatchClauseParameter':CatchClauseParameter2Code,
    'ThrowStatement':ThrowStatement2Code,
    'CatchClause':CatchClause2Code,
    'TryResource':TryResource2Code,
    'This':This2Code,
    'TernaryExpression':TernaryExpression2Code,
    'BreakStatement':BreakStatement2Code
}

In [40]:
len(list(tree))

120

In [436]:
root.types[0].body[0].body[-1].expression.arguments[0].attrs

['condition', 'if_true', 'if_false']

In [ ]:
['prefix_operators',
 'postfix_operators',
 'qualifier',
 'selectors',
 'type',
 'constructor_type_arguments',
 'arguments',
 'body']

In [ ]:
['documentation',
 'modifiers',
 'annotations',
 'type_parameters',
 'return_type',
 'name',
 'parameters',
 'throws',
 'body']

In [548]:
root.types[2].body[2].body[0].expression.value.arguments

[ClassCreator(arguments=[MemberReference(member=stream, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=InputStreamReader, sub_type=None)),
 Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=32768)]

In [337]:
['modifiers',
 'annotations',
 'documentation',
 'name',
 'body',
 'type_parameters',
 'extends',
 'implements']

NameError: name 'modifiers' is not defined

In [184]:
print(ConstructorDeclaration2Code(root.types[0].body[6].body[3]))

public FastReader(String s)throws FileNotFoundException{
br = new BufferedReader(new FileReader(new File(s)));
}



In [166]:
print(MethodDeclaration2Code(root.types[0].body[5].body[8]))

static HashMap<Long,Integer> primeFactorization(long n){
HashMap<Long,Integer> res  = new HashMap<>(); 
while(((n % 2) == 0)){res.put(2L,(res.getOrDefault(2L,0) + 1));
n /= 2;
}for ( long i = 3;((i * i) <= n);i += 2)
{while(((n % i) == 0)){res.put(i,(res.getOrDefault(i,0) + 1));
n /= i;
}}if ( (n > 2))
res.put(n,1);

return res;}



In [42]:
tree2=javalang.parse.parse(AST2Code(root))

In [665]:
tree2=javalang.parse.parse(AST2Code(root))
idx=309 

list_tree=[]
list_tree2=[]
for _,node in tree:
    list_tree.append(node)
for _,node in tree2:
    list_tree2.append(node)

print(list_tree[idx],end='\n\n')
print(list_tree2[idx])
print(str(list_tree[idx]) == str(list_tree2[idx]))

ReturnStatement(expression=ClassCreator(arguments=[MemberReference(member=a, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), MemberReference(member=b, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=[], dimensions=None, name=Pair, sub_type=None)), label=None)

ReturnStatement(expression=ClassCreator(arguments=[MemberReference(member=a, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), MemberReference(member=b, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=Pair, sub_type=None)), label=None)
False


In [ ]:
0 miss match
227 miss match
297 miss match
309 miss match
310 miss match
311 miss match

In [661]:
list_tree2[349].name

'BIT'

In [649]:
root.types[2]

ClassDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=[IfStatement(condition=BinaryOperation(operandl=MemberReference(member=first, postfix_operators=[], prefix_operators=[], qualifier=a, selectors=[]), operandr=MemberReference(member=first, postfix_operators=[], prefix_operators=[], qualifier=b, selectors=[]), operator=!=), else_statement=None, label=None, then_statement=ReturnStatement(expression=BinaryOperation(operandl=Cast(expression=MemberReference(member=first, postfix_operators=[], prefix_operators=[], qualifier=a, selectors=[]), type=BasicType(dimensions=[], name=int)), operandr=Cast(expression=MemberReference(member=first, postfix_operators=[], prefix_operators=[], qualifier=b, selectors=[]), type=BasicType(dimensions=[], name=int)), operator=-), label=None)), IfStatement(condition=BinaryOperation(operandl=MemberReference(member=second, postfix_operators=[], prefix_operators=[], qualifier=a, selectors=[]), operandr=MemberReference(member=second, postfi

In [662]:
equal_test(tree)

0 miss match
227 miss match
297 miss match
309 miss match
310 miss match
311 miss match


False

In [666]:
print(AST2Code(root))

import java.util.*;
import java.io.*;
public  class Main{
static  class FastReader{
BufferedReader br ;
StringTokenizer st ;
public FastReader(){
br = new BufferedReader(new InputStreamReader(System.in));
}
String next(){
while(((st == null) || !st.hasMoreElements())){try{st = new StringTokenizer(br.readLine());

}catch (IOException e){
e.printStackTrace();
}
}return st.nextToken();}
int nextInt(){
return Integer.parseInt(next());}
long nextLong(){
return Long.parseLong(next());}
double nextDouble(){
return Double.parseDouble(next());}
float nextFloat(){
return Float.parseFloat(next());}
boolean nextBoolean(){
return Boolean.parseBoolean(next());}
String nextLine(){
String str  = ""; 
try{str = br.readLine();

}catch (IOException e){
e.printStackTrace();
}
return str;}
}static long modExp(long x,long n,long mod){
long result  = 1; 
while((n > 0)){if ( ((n % 2) == 1))
result = ((((result % mod) * x) % mod) % mod);

x = ((((x % mod) * x) % mod) % mod);
n = (n / 2);
}return result;}
stati

In [5]:
def equal_test(tree1):

    list_tree=[]
    list_tree2=[]
    root=tree.__iter__().__next__()[1]
    tree2=javalang.parse.parse(AST2Code(root))
    for _,node in tree1:
        list_tree.append(node)
    for _,node in tree2:
        list_tree2.append(node)
    
    if len(list_tree) != len(list_tree2):
        print('length miss match!')
        return False
    match=True
    for idx in range(len(list_tree)):
        if str(list_tree[idx]) != str(list_tree2[idx]):
            print(f'{idx } miss match')
            match=False
    if match:
        return True
    else:
        return False
            
    

In [6]:
def AST2Code(root):
    code=''
    for import_item in root.imports:
        code+= 'import '+('static ' if import_item.static else '') +import_item.path+ ('.*' if import_item.wildcard else '') + ';\n'
    childs=root.types
    for child in root.types:
        code+=func_dict[type(child).__name__](child)
    return code

In [7]:
def Nonetype(node):
    return ''

In [72]:
def FieldDeclaration2Code(node):
    code_snippet=''
    for modifier in node.modifiers:
        code_snippet+=modifier+' '
    code_snippet+=Type2Code(node.type)+' '
    for declarator in node.declarators:
        code_snippet+= func_dict[type(declarator).__name__](declarator)+','
    code_snippet=code_snippet.rstrip(',')

    return code_snippet+';\n'

In [9]:
def WhileStatement2Code(node):
    code_snippet='while('+func_dict[type(node.condition).__name__](node.condition)+')'
    code_snippet+=func_dict[type(node.body).__name__](node.body)
    return code_snippet

In [181]:
def ConstructorDeclaration2Code(node):
    code_snippet=''
    for modifier in node.modifiers:
        code_snippet+=' '+modifier
    code_snippet+=' '+ node.name
    code_snippet=code_snippet.lstrip()
    paramter=''
    for pram in node.parameters:
        paramter+=pram.type.name + ('[]'*len(pram.type.dimensions)) +' '+ pram.name +','
    code_snippet+='('+paramter.rstrip(',')+')'
    code_snippet+= ('throws ' +','.join(node.throws) if node.throws else '')
    code_snippet+='{\n'
    for child in node.body:
        code_snippet+= func_dict[type(child).__name__](child)
    return code_snippet+'}\n'

In [659]:
def ClassDeclaration2Code(node):
    code_snippet=(node.documentation+'\n' if node.documentation else '')
    
    for modifier in node.modifiers:
        code_snippet+=modifier+' ' 
    code_snippet+=' class '+node.name
    if node.type_parameters:
        code_snippet+='<'
        for param in node.type_parameters:
            code_snippet+=param.name+','
        
        code_snippet=code_snippet.rstrip(',') +'>'
    if node.implements:
        code_snippet+=' implements '
        for implement in node.implements:
            code_snippet+=Type2Code(implement)
    code_snippet+='{\n'
    
    for class_item in node.body:
        code_snippet+=func_dict[type(class_item).__name__](class_item)

    return code_snippet+"}"

In [609]:
def MethodDeclaration2Code(node):
    code_snippet=(node.documentation+'\n' if node.documentation else '')
    if node.annotations:
        for annotation in node.annotations:
            code_snippet+='@'+annotation.name+'\n'
    for modifier in node.modifiers:
        code_snippet+=modifier+' ' 
    if node.type_parameters:
        code_snippet+='<'
        for type_param in node.type_parameters:
            code_snippet+=type_param.name+','
        code_snippet=code_snippet.rstrip(',')+'> '
        
    code_snippet+=(Type2Code(node.return_type) if node.return_type else 'void')+' '+node.name+'('
    paramter=''
    for pram in node.parameters:
        paramter+=pram.type.name + ('[]'*len(pram.type.dimensions)) +' '+ pram.name +','

    code_snippet+= paramter.rstrip(',')+')'+('throws ' +','.join(node.throws) if node.throws else '')+'{\n'

    
    for method_item in node.body:
        code_snippet+=func_dict[type(method_item).__name__](method_item)
    return code_snippet+'}\n'


In [13]:
def LocalVariableDeclaration2Code(node):
    code_snippet=Type2Code(node.type)+' '
    for declarator in node.declarators:
        code_snippet += func_dict[type(declarator).__name__](declarator) +','
    code_snippet=code_snippet.rstrip(',')
    return code_snippet+'; \n'

In [14]:
def VariableDeclarator2Code(node):
    code_snippet=node.name
    code_snippet+=('[]'*len(node.dimensions) if node.dimensions else '')+' ' 
    
    if node.initializer:
        code_snippet+= ' = '+func_dict[type(node.initializer).__name__](node.initializer)
    return code_snippet#+';\n'

In [257]:
def VariableDeclaration2Code(node):

    code_snippet=''
    code_snippet+=Type2Code(node.type)+' '
    operandr=''
    operandl=''

    for declarator in node.declarators:
        operandr+=declarator.name +','
        operandl += func_dict[type(declarator.initializer).__name__](declarator.initializer)+','

    code_snippet +=operandr.rstrip(',')+' = '+operandl.rstrip(',')
    return code_snippet#+';\n'
    

In [582]:
def Type2Code(node):
    # print(node)
    if type(node).__name__ == 'BasicType':
        return node.name +('[]'*len(node.dimensions) if node.dimensions else '')
    elif type(node).__name__ == 'ReferenceType':
        code_snippet=node.name +('[]'*len(node.dimensions) if node.dimensions else '')
        # if node.name in ['HashMap','HashSet']:
        if node.arguments:
            code_snippet+='<'
            for arg in node.arguments:
                if arg.pattern_type:
                    code_snippet+=arg.pattern_type+','
                if arg.type:
                    code_snippet+=Type2Code(arg.type)+','
            code_snippet=code_snippet.rstrip(',')+'>'

        return code_snippet

In [549]:
def ClassCreator2Code(node):# 미완
    
    argments='('
    for arg in node.arguments:
        argments+=func_dict[type(arg).__name__](arg)+','
    
    argments=argments.rstrip(',')+')'
    code_snippet='new '+Type2Code(node.type)+argments
    return code_snippet

In [88]:
def MemberReference2Code(node):
    
    code_snippet=(''.join(node.prefix_operators) if node.prefix_operators else '')+(node.qualifier+'.' if node.qualifier else '')+node.member 
    selector=''
    if node.selectors:
        for index in node.selectors:
            selector+=func_dict[type(index).__name__](index)
    code_snippet+=selector
    return code_snippet+(''.join(node.postfix_operators) if node.postfix_operators else '')

In [275]:
def ForStatement2Code(node):
    code_snippet='for ( '
    control= node.control
    if node.control.init:
        if type(node.control.init) == list:
            code_snippet+= ''.join([func_dict[type(i).__name__](i).rstrip(';\n') for i in node.control.init])
        else:
            code_snippet+=func_dict[type(node.control.init).__name__](node.control.init).rstrip(';\n')
    
    code_snippet+=';'
    if node.control.condition:
        code_snippet+=func_dict[type(node.control.condition).__name__](node.control.condition)
    code_snippet +=';'
    if node.control.update:
        for update in node.control.update:
            code_snippet+=func_dict[type(update).__name__](update)
    code_snippet+= ')\n'
    
    code_snippet+= func_dict[type(node.body).__name__](node.body)
        
    return code_snippet

In [20]:
def IfStatement2Code(node):
    code_snippet='if ( '
    code_snippet+= func_dict[type(node.condition).__name__](node.condition)+')'
    code_snippet+='\n'+func_dict[type(node.then_statement).__name__](node.then_statement)+'\n'
    if node.else_statement:
        code_snippet+='else '+func_dict[type(node.else_statement).__name__](node.else_statement)
    return code_snippet
    
    

In [21]:
def BlockStatement2Code(node):
    code_snippet="{"
    for block_item in node.statements:
        code_snippet+=func_dict[type(block_item).__name__](block_item)
    code_snippet+="}"
    return code_snippet
    

In [22]:
def BinaryOperation2Code(node):
    code_snippet=func_dict[type(node.operandl).__name__](node.operandl)
    code_snippet+=' ' +node.operator +' '
    code_snippet+=func_dict[type(node.operandr).__name__](node.operandr)
    return '('+code_snippet+')'

In [481]:
def Literal2Code(node):

    prefix=''.join(node.prefix_operators if node.prefix_operators else '')
    postfix=''.join(node.postfix_operators if node.postfix_operators else '')

    return prefix+node.value+postfix

In [24]:
def ArrayCreator2Code(node):
    code_snippet='new '
    code_snippet+= Type2Code(node.type)
    for dim in node.dimensions:
        code_snippet+= '['+func_dict[type(dim).__name__](dim)+']'
    return code_snippet

In [25]:
def Cast2Code(node):
    code_snippet='('+Type2Code(node.type)+')'
    code_snippet+=func_dict[type(node.expression).__name__](node.expression)
    return code_snippet

In [26]:
def StatementExpression2Code(node):
    code_snippet=''
    # print(node)
    code_snippet+=func_dict[type(node.expression).__name__](node.expression)
    return code_snippet+';\n'

In [27]:
def Assignment2Code(node):
    code_snippet=func_dict[type(node.expressionl).__name__](node.expressionl)

    code_snippet+=' ' +node.type+' '
    code_snippet+= func_dict[type(node.value).__name__](node.value)
    return code_snippet

In [529]:
def MethodInvocation2Code(node):
    code_snippet=(node.qualifier+'.' if node.qualifier else '')
    code_snippet+=node.member+'('
    args=''
    
    for arg in node.arguments:
        args+=func_dict[type(arg).__name__](arg)+','
    code_snippet+=args.rstrip(',')+')'
    if node.selectors:
        for s in node.selectors:
            if not type(s).__name__ in ['ArraySelector']:
                code_snippet+= '.'
            code_snippet+=func_dict[type(s).__name__](s)
    prefix_operators=node.prefix_operators if node.prefix_operators else []
    postfix_operators=node.postfix_operators if node.postfix_operators else []
    
    return ''.join(prefix_operators)+code_snippet+''.join(postfix_operators)

In [29]:
def ArraySelector2Code(node):
    code_snippet='['
    code_snippet+=func_dict[type(node.index).__name__](node.index)
    code_snippet+=']'
    return code_snippet

In [30]:
def ReturnStatement2Code(node):
    return 'return '+func_dict[type(node.expression).__name__](node.expression)+';'

In [31]:
def TryStatement2Code(node):
    code_snippet='try'
    if node.resources:
        code_snippet+='('
        for resource in node.resources:
            code_snippet+=func_dict[type(resource).__name__](resource)
        code_snippet+=')'
        
    code_snippet+='{'
    for b in node.block:
        code_snippet+=func_dict[type(b).__name__](b)
    code_snippet+='\n}'
    if node.catches:
        for catch in node.catches:
            code_snippet+= func_dict[type(catch).__name__](catch)
    return code_snippet

In [32]:
def CatchClause2Code(node):

    code_snippet= 'catch ('+func_dict[type(node.parameter).__name__](node.parameter) +')'
    code_snippet+='{\n'
    for catch_block in node.block:
        code_snippet+=func_dict[type(catch_block).__name__](catch_block)
    code_snippet+='}\n'

    return code_snippet

In [33]:
def CatchClauseParameter2Code(node):
    code_snippet=''
    for p in node.types:
        code_snippet+=p+' '
    code_snippet+=node.name
    return code_snippet

In [34]:
def ThrowStatement2Code(node):
    code_snippet='throw ('
    code_snippet+=func_dict[type(node.expression).__name__](node.expression)
    code_snippet+=')'
    return code_snippet +';'

In [36]:
def TryResource2Code(node):
    code_snippet=Type2Code(node.type)+' '
    code_snippet+= node.name

    code_snippet+='='+func_dict[type(node.value).__name__](node.value)
    return code_snippet
    

In [91]:
def This2Code(node):
    code_snippet='this'
    for s in node.selectors:
        code_snippet+= '.'+func_dict[type(s).__name__](s)
    return code_snippet

In [450]:
def TernaryExpression2Code(node):
    code_snippet='('
    if node.condition:
        code_snippet+=func_dict[type(node.condition).__name__](node.condition)
    code_snippet+='?'
    if node.if_true:
        code_snippet+=func_dict[type(node.if_true).__name__](node.if_true)
    code_snippet+=':'
    if node.if_false:
        code_snippet+=func_dict[type(node.if_false).__name__](node.if_false)
    code_snippet+=')'
    return code_snippet
    

In [466]:
def BreakStatement2Code(node):
    code_snippet='break'
    return code_snippet+';'